In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Load housing data from URL to a DataFrame
url = "https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch10/housing.data.txt"
df = pd.read_csv(filepath_or_buffer=url, header=None, sep="\s+")
# Assign the column names
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
# Have a look at the data shape and entries
# Showing the first 5 rows with .head()
# Remove .head() to show all the entries
print(df.shape)
df.head()



In [ ]:
X_multi = df[['RM', 'LSTAT' ]].values
y = df['MEDV'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_multi,y, test_size=0.3, random_state=55)
print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ", y_test.shape)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
sc_x.fit(X_train)
X_train_sc = sc_x.transform(X_train)
X_test_sc = sc_x.transform(X_test)

sc_y.fit(y_train[:, np.newaxis])
y_train_sc = sc_y.transform(y_train[:, np.newaxis])
y_test_sc = sc_y.transform(y_test[:, np.newaxis])

print("X_train_sc.shape: ", X_train_sc.shape)
print("y_train_sc.shape: ", y_train_sc.shape)
print("X_test_sc.shape: ", X_test_sc.shape)
print("y_test_sc.shape: ", y_test_sc.shape)


In [ ]:
from sklearn.kernel_approximation import RBFSampler
rbf_feature = RBFSampler(gamma=0.5, random_state=10)
X_train_sc_feature = rbf_feature.fit_transform(X_train_sc)
X_test_sc_feature = rbf_feature.fit_transform(X_test_sc)

print("X_train_sc_feature.shape: ", X_train_sc_feature.shape)
print("X_test_sc_feature.shape: ", X_test_sc_feature.shape)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
lr = Ridge(alpha=0.5, random_state = 200)
lr.fit(X_train_sc_feature, y_train_sc)
y_pred = lr.predict(X_test_sc_feature)

y_pred = sc_y.inverse_transform(y_pred)
print("MSE: %.3f" %(mean_squared_error(y_test, y_pred)))

In [ ]:
# Data points for visualization
# axis is same
x_axis = X_multi[:, 1] 
y_axis = X_multi[:, 0]
z_axis = y
print("x_axis.shape: ", x_axis.shape)
print("y_axis.shape: ", y_axis.shape)
print("z_axis.shape: ", z_axis.shape)

# Create coordinate matrices for x and y
x_pred = np.linspace(x_axis.min(), x_axis.max(), 30)
print("x_pred.shape: ",x_pred.shape)
y_pred = np.linspace(y_axis.min(), y_axis.max(), 30)
xx_pred, yy_pred = np.meshgrid(x_pred, y_pred) # create meshgrid of points
print("xx_pred.shape: ", xx_pred.shape)
model_viz = np.array([xx_pred.flatten(), yy_pred.flatten()]).T # model_viz.shape: (900,2)
print("model_viz.shape: ", model_viz.shape)
#model_viz = sc_x.transform(model_viz)
model_viz_Quad_2 = rbf_feature.fit_transform(model_viz)  # model_viz_Quad.shape: (900,100)
print("model_viz_Quad_2.shape: ", model_viz_Quad_2.shape)
z_pred = lr.predict(model_viz_Quad_2) # (900,2) -> (900,1)
print("z_pred.shape: ", z_pred.shape)
z_pred = sc_y.inverse_transform(z_pred)
fig = plt.figure(figsize=(18, 6))
ax1 = fig.add_subplot(131, projection='3d')
ax2 = fig.add_subplot(132, projection='3d')
ax3 = fig.add_subplot(133, projection='3d')

axes = [ax1, ax2, ax3]
for ax in axes:
  ax.plot(x_axis, y_axis, z_axis, color='green', zorder=15, linestyle='none',marker='o', alpha=0.5)
  ax.scatter(xx_pred.flatten(), yy_pred.flatten(), z_pred, facecolor=(0,0,0,0), s=20, edgecolor='skyblue')
  ax.set_xlabel('LSTAT', fontsize=10)
  ax.set_ylabel('RM', fontsize=10)
  ax.set_zlabel('MEDV', fontsize=10)
  # ax.locator_params(nbins=4, axis='x')
  # ax.locator_params(nbins=5, axis='x')
ax1.view_init(elev=70, azim=170)
ax2.view_init(elev=30, azim=125)
ax3.view_init(elev=5, azim=120)
fig.tight_layout()